$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
$$
# ACIS
## Architecture, Compiler and Instruction Set
<a id=ACIS></a>
Detector for instruction set, architecture and compiler on raw binaries.


![ACIS Logo](./jupyter_utils/ACIS_Logo.png)

## Introduction


## Contents
- [Part1: ACISDetector With CSV](#ACISDetectorCSV)
- [Part 2: ACISDetector On Raw Binaries](#ACISDetectorRaw)
- [Part 3 : Results and Summary](#Results_Summary)